# 第十二次课 LLM API使用开发作业

学号：

姓名：

# 使用智谱 GLM

智谱 AI 是由清华大学计算机系技术成果转化而来的公司，致力于打造新一代认知智能通用模型。公司合作研发了双语千亿级超大规模预训练模型 GLM-130B，并构建了高精度通用知识图谱，形成数据与知识双轮驱动的认知引擎，基于此模型打造了 ChatGLM（chatglm.cn）。

ChatGLM 系列模型，包括 ChatGLM-130B、ChatGLM-6B 和 ChatGLM2-6B（ChatGLM-6B 的升级版本）模型，支持相对复杂的自然语言指令，并且能够解决困难的推理类问题。其中，ChatGLM-6B 模型来自 Huggingface 上的下载量已经超过 300w（截至 2023 年 6 月 24 日统计数据），该模型在 Hugging Face (HF) 全球大模型下载榜中连续 12 天位居第一名，在国内外的开源社区中产生了较大的影响。

### 1. API 申请指引
首先进入到 [智谱AI开放平台](https://open.bigmodel.cn/overview)，点击`开始使用`或者`开发工作台`进行注册

新注册的用户可以免费领取有效期 1 个月的 100w token 的体验包，进行个人实名认证后，还可以额外领取 400w token 体验包。智谱 AI 提供了 GLM-4 和 GLM-3-Turbo 这两种不同模型的体验入口，可以点击`立即体验`按钮直接体验。

对于需要使用 API key 来搭建应用的话，需要点击右侧的`查看 API key`按钮，就会进入到我们个人的 API 管理列表中。在该界面，就可以看到我们获取到的 API 所对应的应用名字和 `API key` 了。

我们可以点击 `添加新的 API key` 并输入对应的名字即可生成新的 API key。

### 2 调用智谱 GLM API

智谱 AI 提供了 SDK 和原生 HTTP 来实现模型 API 的调用，建议使用 SDK 进行调用以获得更好的编程体验。

首先我们需要配置密钥信息，将前面获取到的 `API key` 设置到 `.env` 文件中的 `ZHIPUAI_API_KEY` 参数，然后运行以下代码加载配置信息。

In [ ]:
import os

from dotenv import load_dotenv, find_dotenv

# 读取本地/项目的环境变量。

# find_dotenv() 寻找并定位 .env 文件的路径
# load_dotenv() 读取该 .env 文件，并将其中的环境变量加载到当前的运行环境中  
# 如果你设置的是全局的环境变量，这行代码则没有任何作用。
_ = load_dotenv(find_dotenv())

智谱的调用传参和其他类似，也需要传入一个 messages 列表，列表中包括 role 和 prompt。我们封装如下的 `get_completion` 函数，供后续使用。

In [ ]:
from zhipuai import ZhipuAI

client = ZhipuAI(
    api_key=os.environ["ZHIPUAI_API_KEY"]
)

def gen_glm_params(prompt):
    '''
    构造 GLM 模型请求参数 messages

    请求参数：
        prompt: 对应的用户提示词
    '''
    messages = [{"role": "user", "content": prompt}]
    return messages


def get_completion(prompt, model="glm-4", temperature=0.95):
    '''
    获取 GLM 模型调用结果

    请求参数：
        prompt: 对应的提示词
        model: 调用的模型，默认为 glm-4，也可以按需选择 glm-3-turbo 等其他模型
        temperature: 模型输出的温度系数，控制输出的随机程度，取值范围是 0~1.0，且不能设置为 0。温度系数越低，输出内容越一致。
    '''

    messages = gen_glm_params(prompt)
    response = client.chat.completions.create(
        model=model,
        messages=messages,
        temperature=temperature
    )
    if len(response.choices) > 0:
        return response.choices[0].message.content
    return "generate answer error"

In [ ]:
get_completion("你好")

### 3 构建知识库

3.1 读取数据文件

In [ ]:
file_paths = []
folder_path = './data_base/papers'
for root, dirs, files in os.walk(folder_path):
    for file in files:
        file_path = os.path.join(root, file)
        file_paths.append(file_path)
print(file_paths[:2])

In [ ]:
from langchain.document_loaders.pdf import PyMuPDFLoader
from langchain.document_loaders.markdown import UnstructuredMarkdownLoader

# 遍历文件路径并把实例化的loader存放在loaders里
loaders = []

for file_path in file_paths:

    file_type = file_path.split('.')[-1]
    if file_type == 'pdf':
        loaders.append(PyMuPDFLoader(file_path))
    elif file_type == 'md':
        loaders.append(UnstructuredMarkdownLoader(file_path))

In [ ]:
# 下载文件并存储到text
texts = []

for loader in loaders: texts.extend(loader.load())

载入后的变量类型为`langchain_core.documents.base.Document`, 文档变量类型同样包含两个属性
- `page_content` 包含该文档的内容。
- `meta_data` 为文档相关的描述性数据。

In [ ]:
text = texts[1]
print(f"每一个元素的类型：{type(text)}.", 
    f"该文档的描述性数据：{text.metadata}", 
    f"查看该文档的内容:\n{text.page_content[0:]}", 
    sep="\n------\n")

In [ ]:
from langchain.text_splitter import RecursiveCharacterTextSplitter

# 切分文档
text_splitter = RecursiveCharacterTextSplitter(
    chunk_size=500, chunk_overlap=50)

split_docs = text_splitter.split_documents(texts)

In [ ]:
len(split_docs)

### 3.2 构建Chroma向量库

Langchain 集成了超过 30 个不同的向量存储库。我们选择 Chroma 是因为它轻量级且数据存储在内存中，这使得它非常容易启动和开始使用。

LangChain 可以直接使用 OpenAI 和百度千帆的 Embedding，同时，我们也可以针对其不支持的 Embedding API 进行自定义，例如，我们可以基于 LangChain 提供的接口，封装一个 zhupuai_embedding，来将智谱的 Embedding API 接入到 LangChain 中。在本章的[附LangChain自定义Embedding封装讲解](./附LangChain自定义Embedding封装讲解.ipynb)中，我们以智谱 Embedding API 为例，介绍了如何将其他 Embedding API 封装到 LangChain
中，欢迎感兴趣的读者阅读。

**注：如果你使用智谱 API，你可以参考讲解内容实现封装代码，也可以直接使用我们已经封装好的代码[zhipuai_embedding.py](./zhipuai_embedding.py)，将该代码同样下载到本 Notebook 的同级目录，就可以直接导入我们封装的函数。在下面的代码 Cell 中，我们默认使用了智谱的 Embedding，将其他两种 Embedding 使用代码以注释的方法呈现，如果你使用的是百度 API 或者 OpenAI API，可以根据情况来使用下方 Cell 中的代码。**

In [ ]:
# 使用 OpenAI Embedding
# from langchain.embeddings.openai import OpenAIEmbeddings
# 使用百度千帆 Embedding
# from langchain.embeddings.baidu_qianfan_endpoint import QianfanEmbeddingsEndpoint
# 使用我们自己封装的智谱 Embedding，需要将封装代码下载到本地使用
from zhipuai_embedding import ZhipuAIEmbeddings

# 定义 Embeddings
# embedding = OpenAIEmbeddings() 
embedding = ZhipuAIEmbeddings()
# embedding = QianfanEmbeddingsEndpoint()

# 定义持久化路径
persist_directory = './data_base/vector_db/chroma_papers'

In [ ]:
from langchain.vectorstores.chroma import Chroma

vectordb = Chroma.from_documents(
    documents=split_docs[:20], # 为了速度，只选择前 20 个切分的 doc 进行生成；使用千帆时因QPS限制，建议选择前 5 个doc
    embedding=embedding,
    persist_directory=persist_directory  # 允许我们将persist_directory目录保存到磁盘上
)

In [ ]:
vectordb.persist()

In [ ]:
print(f"向量库中存储的数量：{vectordb._collection.count()}")

### 3.3 向量检索
#### 3.3.1 相似度检索

In [ ]:
question="could you explain in-context learning"

sim_docs = vectordb.similarity_search(question,k=3)
print(f"检索到的内容数：{len(sim_docs)}")

In [ ]:
for i, sim_doc in enumerate(sim_docs):
    print(f"检索到的第{i}个内容: \n{sim_doc.page_content}", end="\n--------------\n")

#### 3.3.2 MMR检索
如果只考虑检索出内容的相关性会导致内容过于单一，可能丢失重要信息。

最大边际相关性 (`MMR, Maximum marginal relevance`) 可以帮助我们在保持相关性的同时，增加内容的丰富度。

核心思想是在已经选择了一个相关性高的文档之后，再选择一个与已选文档相关性较低但是信息丰富的文档。这样可以在保持相关性的同时，增加内容的多样性，避免过于单一的结果。

In [ ]:
mmr_docs = vectordb.max_marginal_relevance_search(question,k=3)

for i, sim_doc in enumerate(mmr_docs):
    print(f"MMR 检索到的第{i}个内容: \n{sim_doc.page_content[:200]}", end="\n--------------\n")